## Naive Retrieval-Augmented Generation

Для построения конвейера RAG используем библиотеку [LangChain](https://www.langchain.com/), а для доступа к YandexGPT - [yandex_chain](https://github.com/yandex-datasphere/yandex_chain).

Мы предполагаем, что параметры для вызова YandexGPT - `folder_id` и `api_key` - уже установлены в переменных окружения.

Для начала разархивируем необходимые файлы и установим библиотеки нужных версий:

In [1]:
!unzip -q chroma_db.zip
!unzip -q chroma_hypoq_db.zip

In [2]:
%pip install -q langchain==0.2.8 langchain-chroma==0.1.2 chromadb==0.4.18
%pip install -q --no-deps yandex-chain==0.0.9

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress t

Научимся вычислять эмбеддинги фрагментов текста:

In [1]:
import os
from yandex_chain import YandexEmbeddings

embeddings = YandexEmbeddings(
    folder_id=os.environ['folder_id'], 
    api_key=os.environ['api_key'])

vec = embeddings.embed_query("Hello, world!")
len(vec)

256

Поскольку наш текст содержит ударения в тексте, а также лишние кавычки, напишем функцию для удаления этого:

In [2]:
import unicodedata

ACCENT_MAPPING = {
    '́': '',
    '̀': '',
    'а́': 'а',
    'а̀': 'а',
    'е́': 'е',
    'ѐ': 'е',
    'и́': 'и',
    'ѝ': 'и',
    'о́': 'о',
    'о̀': 'о',
    'у́': 'у',
    'у̀': 'у',
    'ы́': 'ы',
    'ы̀': 'ы',
    'э́': 'э',
    'э̀': 'э',
    'ю́': 'ю',
    '̀ю': 'ю',
    'я́́': 'я',
    'я̀': 'я',
}
ACCENT_MAPPING = {unicodedata.normalize('NFKC', i): j for i, j in ACCENT_MAPPING.items()}


def unaccentify(s):
    source = unicodedata.normalize('NFKC', s)
    for old, new in ACCENT_MAPPING.items():
        source = source.replace(old, new)
    return source

def normalize(text):
    return (unaccentify(text)
            .replace('«','')
            .replace('»','')
            .replace('"','')
            .replace('<','')
            .replace('>',''))


Собираем все фрагменты текста по винам и регионам в соответствующие переменные:

In [3]:
with open('../source/wines.txt',encoding='utf-8') as f:
    wines = ''.join(f.readlines())
with open('../source/regions.txt',encoding='utf-8') as f:
    regions = ''.join(f.readlines())

wines = [normalize(x) for x in wines.split('-----')]
regions = [normalize(x) for x in regions.split('-----')]

Посмотрим, можем ли мы использовать длинные фрагменты текста целиком. Посмотрим на максимальную длину фрагментов:

In [4]:
max(len(x) for x in wines+regions)

2852

Это меньше, чем 2500 токенов (лимит для вычисления эмбеддингов), поэтому будем класть фрагменты в векторную базу целиком. В качестве векторной базы используем ChromaDB.

> **ВНИМАНИЕ**: Код ниже вычисляет эмбеддинги для всех фрагментов текста. Это может занять несколько минут. Мы в репозитории предоставили уже предвычисленные эмбеддинги: вы можете пропустить следующую ячейку, и воспользоваться уже готовыми эмбеддингами.

> **ВНИМАНИЕ**: Выполните либо следующую ячейку, и пропустите ячейку после неё, либо выполните ячейку через одну, чтобы воспользоваться готовой БД.

In [4]:
from langchain_chroma import Chroma

db = Chroma.from_texts(wines+regions, embeddings, persist_directory='./chroma_db')

Загружаем базу данных для дальнейшего использования:

In [5]:
from langchain_chroma import Chroma
db = Chroma(embedding_function=embeddings, persist_directory='./chroma_db')

Для поиска фрагментов создадим объект `retriever`. В качестве параметра передаём количество фрагментов и тип поиска. MMR-поиск (Maximal Marginal Relevance) позволяет исключить из выдачи слишком похожие фрагменты.

In [6]:
q = "Что едят с мерло?"
retriever = db.as_retriever(search_type="mmr", search_kwargs={"k": 5})
res = retriever.invoke(q)
res

[Document(page_content='\nМерло\nМерло (фр. Merlot) — сорт винного красного винограда из Бордо, винодельческого региона Франции. Название сорта происходит от merle (фр.) — черный дрозд, обозначая сходство темного оттенка ягод с оперением птиц. Мерло растет в умеренном и теплом климате и устойчив к холодам. Этот сорт прост в уходе, обладает высокой урожайностью, что делает его вторым по распространенности красным виноградом в мире — после каберне совиньон. Мерло выращивают во Франции, США, Италии и других странах.\nСамое раннее упоминание мерло относится к 1784 году: чиновник из Бордо назвал вино из этого сорта лучшим из представленных правобережными виноградниками Либурне. К XIX веку виноград регулярно сажали в соседнем северном регионе Медок на левом берегу Жиронды.\nИз мерло получаются мягкие вина с низкой кислотностью, минимальным содержанием танинов, бархатистым вкусом. Мерло используют в красных и розовых блендах вин, чтобы сбалансировать сухость, крепость и терпкие танинные свойс

Для получения естественного ответа на вопрос необходимо построить цепочку, которая сначала вызывает `retriever` для извлечения релевантных фрагментов текста, а затем забрасывает релевантные фрагменты в контекст модели, и просит её ответить на исходный вопрос с учётом контекста.

In [7]:
import langchain.chains
import langchain.prompts
from yandex_chain import YandexLLM, YandexGPTModel
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

llm = YandexLLM(folder_id=os.environ['folder_id'],
                api_key=os.environ['api_key'],
                model=YandexGPTModel.Pro)

prompt = """
Пожалуйста, посмотри на текст ниже и ответь на вопрос, используя информацию из этого текста. Выведи только
краткий ответ, не надо пояснительного текста.
Текст:
-----
{context}
-----
Вопрос:
{question}"""

prompt = langchain.prompts.PromptTemplate(
    template=prompt, input_variables=["context", "question"]
)

def join_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Создаём цепочку
chain = (
    {"context": retriever | join_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

chain.invoke(q)

'С мерло едят колбасу, сыр средней выдержки, мясные супы, горячие блюда из мяса, птицы и красной рыбы.'

In [8]:
chain.invoke('Какие существуют итальянские вина?')

'Существуют такие итальянские вина, как: амароне, соаве, просекко, речото, вальполичелла, бардолино,  бароло, Барбареско, гави, Asti, марсала, черасуоло ди виттория, биферно и многие другие.'

In [9]:
chain.invoke('Какие вина из Южной Африки лучше пить с сыром?')

'Пинотаж.'

## Стратегия с гипотетическими вопросами

Проблема с наивным RAG в том, что вектор эмбеддинга вычисляется по достаточно длинному фрагменту текста. Чтобы это исправить, можно по фрагментам текста получать более короткие и ёмкие по содержанию под-фрагменты (например, используя суммаризацию), или гипотетические вопросы. В этом случае мы можем добиться более точного попадания по смыслу в процессе поиска.

Реализуем подход с гипотетическими вопросами. Для этого используем LLM для генерации гипотетических вопросов к тексту.

In [10]:
extract_q_prompt = """
## Задача
Ты - литературный редактор, задача которого - придумать как можно больше вопросов к заданному тексту.
На вход поступает фрагмент текста, твоя задача - вернуть список вопросов к тесту. Каждый вопрос пиши
в скобках с новой строки. Не пиши никакого другого текста, кроме вопросов. Старайся избегать слов
"этот", "это" в вопросах, например, вместо "Когда этот сорт стали производить" пиши "Когда стали производить
Мерло".

## Пример:
Текст: Сорт винограда Мерло начал производиться во Франции в 1760 году.
Результат:
(В каком году начал производиться сорт винограда Мерло?)
(В какой стране начали производить сорт винограда Мерло?)
(Какой сорт винограда начали производить в 1760 году во Франции?)

## Задание
Текст: {}
Результат:
"""

llm.invoke(extract_q_prompt.format(wines[0]))

'(Какой сорт винограда считается одним из древнейших мировых сортов?)\n(От какого сорта произошли каберне совиньон, карменер и мерло?)\n(Что обозначает слово «фран» в названии сорта винограда?)\n(Для производства каких вин используют каберне фран?) \n\n(На какие сорта делят вина, произведённые из каберне фран?)\n(Какие регионы известны производством вин из каберне фран?)\n(Каким ароматом отличаются красные вина из каберне фран?)\n(Чем могут различаться оттенки вкуса красных вин из этого сорта?)\n(Как производят розовые вина из винограда каберне фран? Какими характеристиками они обладают?)\n(Какими винами может сопровождаться каберне фран и почему?)\n(С какими блюдами сочетаются лёгкие вина из сорта каберне фран?)\n(Какой тип мяса может подаваться с яркими и выразительными винами из винограда каберне фран?)\n(Какие элементы способны подчеркнуть и дополнить вкус вина каберне фран при подаче с едой?)\n(В каком качестве может использоваться вино каберне фран во время приёма пищи?)'

Теперь пройдёмся по всем фрагментам текста и сгенерируем для них гипотетические вопросы. 

> **ВНИМАНИЕ**: Это длительная процедура, поэтому мы приготовили для вас готовый файл с вопросами в директории `hypo_questions`.Вы можете пропустить следующую ячейку.

In [37]:
from tqdm.auto import tqdm
import re
import json

questions = []

def extract_questions(x):
    res = llm.invoke(extract_q_prompt.format(wines[0]))
    res = res.split('\n')
    q = []
    for x in res:
        if z:=re.match(r'\((.*)\)',x):
            z = z.string.strip()[1:-1]
            q.append(z)
    return q

for x in tqdm(wines+regions):
    q = extract_questions(wines[0])
    questions.append({
        "text" : x,
        "questions" : q 
    })

with open('hypo_questions/questions.json','w',encoding='utf-8') as f:
    json.dump(questions,f,indent=4, ensure_ascii=False)

100%|██████████| 235/235 [25:35<00:00,  6.53s/it]


Загрузим гипотетические вопросы с диска.

In [38]:
with open('hypo_questions/questions.json',encoding='utf-8') as f:
    questions = json.load(f)

Теперь добавим гипотетические вопросы в векторную БД. Сам текст будем сохранять в метаданных. Поскольку по каждому фрагменту текста есть много вопросов, и для каждого нужно вычислять эмбеддинг - выполнение следующей ячейки может занять много времени. Мы подготовили для вас готовую базу данных, поэтому следующую ячейку **можно пропустить**.

> **ВНИМАНИЕ**: Выполните либо следующую ячейку для генерации базы данных и пропустите ячейку через одну, либо пропустите следующую ячейку и выполните ячейку через одну, чтобы загрузить готовую базу данных.

In [ ]:
qdb = Chroma(embedding_function=embeddings, persist_directory='./chroma_hypoq_db')
for x in questions:
    if len(x['questions'])>0:
        qdb.add_texts(x['questions'],metadatas=[{ "text" : x['text']}]*len(x['questions']))

In [12]:
qdb = Chroma(embedding_function=embeddings, persist_directory='./chroma_hypoq_db')

Определим `retriever` для доступа к этой векторной БД:

In [12]:
q = "Какие блюда подходят к Каберне Фран?"
retriever = qdb.as_retriever(search_type="mmr", search_kwargs={"k": 10})
res = retriever.invoke(q)
res

[Document(metadata={'text': '\nМозель\nМозель — винодельческий регион Германии, названный от реки Мозель, в долине которой располагается. Славится винами высокого качества из винограда сорта рислинг.\nГлавная особенность Мозеля как винодельческого региона — сланцевые почвы: они хорошо сохраняют тепло и обеспечивают дренаж в случае, когда река выходит из берегов. Минералы, которыми насыщены сланцевые почвы, придают вину ароматы зеленых яблок, персика и лимонов. \nБолее 60% виноградников отданы под рислинг, но в регионе также выращивают пино нуар и мюллер-тургау, выведенный швейцарским ботаником Германом Мюллером. Этот сорт используется для производства столовых вин.\nВ Мозеле производят столовые, сухие, полусухие и десертные вина. \nМозельские белые вина отличаются светло-соломенным цветом, а в их букете чувствуются цветы и полевые травы. Чистые, соленые, хрустящие — самые подходящие эпитеты для вин этого региона. \nТакже в Мозеле производят айсвайн — сладкое вино из замороженного виног

В нашем случае нам нужен более хитрый алгоритм, при котором мы возвращаем не сами найденные вопросы, а соответствующие им тексты. Кроме того, надо учесть, что тексты могут повторяться, поэтому с помощью `set` выберем из них уникальные:

In [13]:
def xretriever(q):
    res = retriever.invoke(q)
    return '\n'.join(set([x.metadata['text'] for x in res]))

print(xretriever(q))


Эмилия-Романья
Эмилия-Романья — регион Италии, который расположен между Лигурийским морем на востоке и Адриатическим на западе. Состоит из двух исторических частей — Эмилии на северо-западе и Романьи на юго-востоке. Область занимает чуть более 7% всей территории страны, являясь одной из крупнейших по размеру в Италии. Административный центр — город Болонья. 

Эмилия-Романья — крупный винодельческий регион Италии, занимающий пятое место в стране по площадям виноградников и третье — по объему винопроизводства. Здесь область уступает лишь Венето и Апулии. 
Две трети территории — это равнины с плодородными и разнообразными почвами. Моря, щедрая река По и щит от холодов в виде Аппенинских гор делают регион благоприятным для сельского хозяйства и виноделия.
Регион славится своим виноградом ламбруско. Для красных вин здесь часто используют ламбруско граспаросса, для розовых и игристых — ламбруско ди сорбара. Всего же известно около 60 сортов этого винограда. 
Из грекетто ди тоди — кислотного

С учётом этого, цепочка RAG будет выглядеть так:

In [14]:
chain = (
    {"context": xretriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

chain.invoke(q)

'Каберне Фран — сорт красного винограда, из которого делают вина во многих винодельческих регионах.\n\n**По общему правилу красные вина подают с мясными блюдами.** Однако многое зависит от конкретного региона производства, сорта винограда и характеристик вина. \nДля более точного ответа на вопрос о сочетании конкретных вин с определёнными блюдами нужна дополнительная информация: название вина, особенности его вкуса и аромата.'

## Заключение

Помимо вопросов, можно использовать другие стратегии выбора, суммаризацию, или разбиение текста на более мелкие фрагменты. В этом случае может оказаться полезным использовать иерархическую БД вроде Neo4j.

В любом случае, при создании RAG-системы необходимо её тщательное тестирование, чтобы убедиться, что находятся релевантные фрагменты текста, и что полученные ответы корректны.